<a href="https://colab.research.google.com/github/SaiRajesh228/DeepLearningAssignment1/blob/main/q11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from keras.datasets import fashion_mnist
import wandb

In [33]:
!pip install wandb -Uq
import wandb
from getpass import getpass

# 1. Authenticate with W&B
wandb.finish()
api_key = getpass("Enter W&B API key: ")
wandb.login(key=api_key, relogin=True)

# 2. Verify authentication and get username
try:
    api = wandb.Api()
    user = api.viewer()  # Get user details
    username = user.get("entity")  # Extract username
    print(f"✓ Authenticated as: {username}")
except Exception as e:
    print(f"❌ Authentication failed: {e}")
    exit(1)

# 3. Sweep Configuration
sweep_config = {
    'method': 'random',
    'metric': {'name': 'accuracy', 'goal': 'maximize'},
    'parameters': {
        'learning_rate': {'values': [1e-3, 1e-4]},
        'batch_size': {'values': [16, 32, 64]},
        'epochs': {'values': [5, 10]},
        'hidden_layers': {'values': [3, 4, 5]},
        'hidden_size': {'values': [32, 64, 128]},
        'activation': {'values': ['relu', 'sigmoid', 'tanh']},
        'optimizer': {'values': ['sgd', 'momentum', 'nesterov', 'rmsprop', 'adam', 'nadam']},
        'weight_init': {'values': ['random', 'xavier']},
        'weight_decay': {"values": [0, 0.0005, 0.5]}
    },
    "run_cap": 100
}

# 4. Create Sweep Under Correct Username
try:
    sweep_id = wandb.sweep(sweep_config, entity=username, project="DLAssignment1")
    print(f"✅ Sweep created! View at: https://wandb.ai/{username}/DLAssignment1/sweeps/{sweep_id}")
except Exception as e:
    print(f"❌ Error: {e}")
    print("Verify:")
    print(f"1. Project exists at https://wandb.ai/{username}/DLAssignment1")
    print("2. API key has write permissions")
finally:
    wandb.finish()

Enter W&B API key: ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


❌ Authentication failed: 'NoneType' object has no attribute 'keys'
❌ Error: name 'username' is not defined
Verify:


NameError: name 'username' is not defined